# Feed Dataset through Queue to Tensorflow from HDFS

## Populate HDFS with Sample Dataset

In [ ]:
%%bash

hadoop fs -copyFromLocal /root/datasets/linear /

In [ ]:
%%bash

hadoop fs -ls /linear

## Create Queue and Feed Tensorflow from HDFS

### Show TensorFlow-HDFS Code

In [ ]:
%%bash

cat /root/src/main/python/queue/tensorflow_hdfs.py

## Open a Terminal through Jupyter Notebook 
### (Menu Bar -> Terminal -> New Terminal)
![Jupyter Terminal](https://s3.amazonaws.com/fluxcapacitor.com/img/jupyter-terminal.png)

## Run the following in the Terminal
```
queue_hdfs
```
_Note:  This will error out in the notebook.  You must run from Terminal!_

In [ ]:
%%bash

cat /root/scripts/queue_hdfs